**Промежуточное представление (IR)** — это представление программы между исходным и целевым языками, которое облегчает анализ и оптимизацию программы для компилятора. Таким образом, при проектировании IR необходимо учитывать сложность преобразования исходного языка в целевой язык, а также простоту использования и производительность анализа и оптимизации программ.

**MindSpore IR `(MindIR)`** — это  IR в функциональном стиле, основанный на графовом представлении. Его основное назначение заключается в обслуживании автоматического дифференциального преобразования. Автоматическое дифференцирование использует метод преобразования, основанный на программном фреймворке функционального стиля. Таким образом, IR использует семантику, близкую к семантике функции ANF.

**ANF Syntax**
```
<aexp> ::= NUMBER | STRING | VAR | BOOLEAN | PRIMOP
          |  (lambda (VAR …) <exp>)
<cexp> ::= (<aexp> <aexp> …)
          |  (if <aexp> <exp> <exp>)
<exp> ::= (let ([VAR <cexp>]) <exp>) | <cexp> | <aexp>
```

Выражения в ANF классифицируются на атомарные выражения (`aexp`) и составные выражения (`cexp`). Атомарное выражение указывает на постоянное значение, переменную или анонимную функцию. Составное выражение состоит из нескольких атомарных выражений, указывающих на то, что вызывается анонимная функция или примитивная функция. Первое входное выражение составного выражения — это вызываемая функция, а остальные входные выражения — вызываемые параметры.

**MindIR Syntax**
```
<ANode> ::= <ValueNode> | <ParameterNode>
<ParameterNode> ::= Parameter
<ValueNode> ::= Scalar | Named | Tensor | Type | Shape
               | Primitive | MetaFuncGraph | FuncGraph
<CNode> ::= (<AnfNode> …)
<AnfNode> ::= <CNode> | <ANode>
```

`ANode` в MindIR соответствует атомной экспрессии ANF. `ANode` имеет два подкласса: `ValueNode` и `ParameterNode`. `ValueNode` относится к постоянному узлу, который может нести постоянное значение (например, скаляр, символ, тензор, тип и размерность), примитивную функцию (`Primitive`), метафункцию (`MetaFuncGraph`) или общую функцию (`FuncGraph`). В функциональном программировании определение функции само по себе является значением. ParameterNode относится к узлу параметра, который указывает на формальный параметр функции.

`CNode` в MindIR соответствует составному выражению ANF, указывающему на вызов функции.

Во время автоматического дифференцирования MindSpore вычисляется вклад градиента ParameterNode и `CNode`, и возвращается итоговый градиент ParameterNode. Градиент `ValueNode` не вычисляется.

### Пример

In [2]:
import mindspore as ms

def func(x, y):
    return x / y

@ms.jit
def test_f(x, y):
    a = x - 1
    b = a + y
    c = b * func(a, b)
    return c


**Как это представляет MindIR**
```
lambda (x, y)
    let a = x - 1 in
    let b = a + y in
    let func = lambda (x, y)
        let ret = x / y in
        ret end in
    let %1 = func(a, b) in
    let c = b * %1 in
    c end
```

**Представление в графическом плане**

![graph](./pictures/MindDir_Graph.png)

## Higher-Order Functions

В MindIR функция определяется с помощью подграфа. Однако сама функция может быть передана в качестве входных данных или выходных данных других функций более высокого порядка. В следующем простом примере функция `f` передается в качестве параметра в функцию `g`. Следовательно, функция `g` является функцией более высокого порядка, которая принимает входные данные функции, и фактическое место вызова функции `f` находится внутри функции `g`.

In [ ]:
@ms.jit
def hof(x):
    def f(x):
        return x + 3
    def g(function, x):
        return function(x) * function(x)
    res = g(f, x)
    return res

![graph](./pictures/MindIR-high-order-function_graph.png)

## Control FLows

В MindIR потоки управления представлены в виде выбора и вызова функций более высокого порядка. Эта форма преобразует поток управления в поток данных функций более высокого порядка, что делает автоматический дифференциальный алгоритм более мощным. Он не только поддерживает автоматическую дифференциацию потоков данных, но и поддерживает автоматическую дифференциацию потоков управления, таких как условные переходы, циклы и рекурсия

In [ ]:
@ms.jit
def fibonacci(n):
    if n < 1:
        return 0
    if n == 1:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)

![graph](./pictures/MindIR-COntrol-Flows.png)

Два графика функций на верхнем уровне выбираются и вызываются с помощью переключателя (`swith`). `✓fibonacci` - это истинная ветвь первого if, а `✗fibonacci` - ложная ветвь первого if. `✓✗fibonacci`, вызываемы в `✗fibonacci`, является истинной ветвью elif, а `✗✗fibonacci` - ложной ветвью elif. Суть в том, что в MindIR условные переходы и рекурсия представлены в виде потоков управления более высокого порядка. Например, `✓✗fibonacci` и `✗fibonacci` передаются в качестве параметров оператора `switch`. `switch` выбирает функцию в качестве возвращаемого значения на основе параметра `condition`. Таким образом, `switch` выполняет операцию двоичного выбора для входных функций в качестве общих значений и не вызывает сами функции. Реальный вызов функции завершается в `CNode`, следующем за `switch`.

## Free Variables and Closures

**Замыкание** — это функция языка программирования, которая относится к комбинации блоков кода и области видимости.

**Свободная переменная** — это переменная, находящаяся в области видимости и используемая в блоке кода вместо локальной переменной.

В MindIR блок кода представлен в виде графа функций. Область видимости можно рассматривать как контекст, в котором вызывается функция. Метод захвата свободных переменных копирует значения, а не передает их по ссылке.


In [ ]:
@ms.jit
def func_outer(a, b):
    def func_inner(c):
        return a + b + c
    return func_inner

@ms.jit
def ms_closure():
    closure = func_outer(1, 2)
    out1 = closure(1)
    out2 = closure(2)
    return out1, out2

import pandas as pd

data = {
    "(32, 1)":7.91,
    "(32, 2)":15.81,
    "(32, 4)":31.62,
    "(32, 8)": 63.25,
    "(32, 16)":126.51,
    "(32, 32)":253.01,
}

pd.DataFrame(data, index=['loss'])

![graph](./pictures/MindIR-free_variables.png)

В примере `a` и `b` являются свободными переменными, поскольку переменные `a` и `b` в `func_inner` являются параметрами, определенными в родительском графе, на который ссылается `func_outer`. Переменная `closure` - это замыкание, представляющее собой комбинацию функции `func_inner` и ее контекстной функции `func_outer(1, 2)`. Следовательно, результат `out1` равен 4, что эквивалентно 1+2+1, а результат `out2` равен 5, что эквивалентно 1+2+2.